5. Convert the ConsensusXML file to a FeatureQuantificationTable.txt file compatible to GNPS

In [1]:
from pyopenms import *
import pandas as pd

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


In [ ]:
input_consensus= "results/GNPSexport/interim/filtered.consensusXML"
cmap = ConsensusMap()
ConsensusXMLFile().load(input_consensus, cmap)

# get intensities as a DataFrame
intensities = cmap.get_intensity_df()

# get meta data as DataFrame
meta_data = cmap.get_metadata_df()

# you can concatenate these two for a "result" DataFrame
result = pd.concat([meta_data, intensities], axis=1)

# if you don't need labeled index, remove it (and/or save with index = False)
result.reset_index(drop=True, inplace=True)

idx = 0
new_col = "CONSENSUS"  # can be a list, a Series, an array or a scalar   
result.insert(loc=idx, column='#CONSENSUS', value=new_col)   
result= result.rename(columns= {"charge": "charge_cf", "RT": "rt_cf", "mz": "mz_cf", "quality": "quality_cf", "width": "width_cf"})
result= result.drop(["sequence"], axis= 1)
result= result.sort_index(axis=1)
result

In [12]:
filemeta= cmap.getColumnHeaders()
mapIDs = [k for k in filemeta.keys()]
filename= []
size=[]
label= []
for header in filemeta.values():
    files= header.filename
    sizes= header.size
    labels= header.label
    filename.append(files)
    size.append(sizes)
    label.append(labels)

dict = {'id': mapIDs, 'filename': filename,'label': label,'size': size}
DF= pd.DataFrame(dict)
idx = 0
new_col = "MAP"  # can be a list, a Series, an array or a scalar   
DF.insert(loc=idx, column='#MAP', value=new_col)   
DF["id"] = "intensity_"+ (DF["id"]).astype(str)
DF

,#MAP,id,filename,label,size
0,MAP,intensity_0,results/interim/PCpeak_20210826_UMETAB219_POS_...,,2776
1,MAP,intensity_1,results/interim/PCpeak_20210826_UMETAB219_POS_...,,1253
2,MAP,intensity_2,results/interim/PCpeak_20210826_UMETAB219_POS_...,,1345
3,MAP,intensity_3,results/interim/PCpeak_20210826_UMETAB219_POS_...,,1353
4,MAP,intensity_4,results/interim/PCpeak_20210826_UMETAB219_POS_...,,3290
5,MAP,intensity_5,results/interim/PCpeak_20210826_UMETAB219_POS_...,,2979
6,MAP,intensity_6,results/interim/PCpeak_20210826_UMETAB219_POS_...,,3070
7,MAP,intensity_7,results/interim/PCpeak_20210826_UMETAB219_POS_...,,3044
8,MAP,intensity_8,results/interim/PCpeak_20210826_UMETAB219_POS_...,,1351
9,MAP,intensity_9,results/interim/PCpeak_20210826_UMETAB219_POS_...,,1026


In [18]:
cols= result.columns
for col in cols:
    for i, path in enumerate(filename):
        if path== col:
            name= DF["id"][i]
            result.rename(columns={col: name}, inplace=True)

result

,#CONSENSUS,charge_cf,mz_cf,quality_cf,intensity_0,intensity_1,intensity_2,intensity_3,intensity_4,intensity_5,...,intensity_39,intensity_40,intensity_41,intensity_42,intensity_43,intensity_44,intensity_45,intensity_46,intensity_47,rt_cf
0,CONSENSUS,1,393.223224,0.000446,7443680.0,13535850.0,13423620.0,14076760.0,30551950.0,22427880.0,...,8631237.0,8919085.0,5242951.0,6956869.0,4781326.0,3228996.0,2087019.0,1.962420e+06,1644674.0,296.581818
1,CONSENSUS,1,379.207510,0.000301,3699400.0,10704690.0,9434231.0,8763564.0,12793220.0,8378248.0,...,4992339.0,2824778.0,2025389.0,2312600.0,2618161.0,3579418.0,4913860.0,4.470871e+06,4262092.0,141.109551
2,CONSENSUS,1,254.161202,0.003344,121834096.0,114376000.0,137001200.0,136163504.0,48087800.0,49578168.0,...,66059480.0,12672740.0,16124980.0,7442163.0,7042243.0,13681260.0,10477380.0,8.470678e+06,9886484.0,79.803092
3,CONSENSUS,1,407.238863,0.000433,13193210.0,17892640.0,19377870.0,13692410.0,19822080.0,15748400.0,...,13708230.0,6982376.0,2733415.0,2697249.0,4937243.0,3818414.0,1094928.0,7.286438e+05,4729806.0,315.842665
4,CONSENSUS,1,328.139062,0.001450,36532420.0,55908592.0,53082512.0,45685432.0,55692720.0,50182540.0,...,7893263.0,19129270.0,8776934.0,9347569.0,10517510.0,42571260.0,13239720.0,1.220015e+07,10398600.0,151.219209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6682,CONSENSUS,1,450.255977,0.000009,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,144.461281
6683,CONSENSUS,1,380.218136,0.000024,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,364.670106
6684,CONSENSUS,1,482.272256,0.000082,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,283.771130
6685,CONSENSUS,1,399.223823,0.000128,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,343.010734


In [19]:
cols = result.columns
preordered = ["#CONSENSUS", "charge_cf", "rt_cf", "mz_cf", "quality_cf", "width_cf"]
new_cols = preordered + [c for c in result.columns if c not in preordered]
new_df = result.reindex(columns=new_cols)
new_df.to_csv('results/GNPSexport/FeatureQuantificationTable.txt', sep = '\t', index = False)
print(new_df)

     #CONSENSUS  charge_cf       rt_cf       mz_cf  quality_cf  width_cf  \
0     CONSENSUS          1  296.581818  393.223224    0.000446       NaN   
1     CONSENSUS          1  141.109551  379.207510    0.000301       NaN   
2     CONSENSUS          1   79.803092  254.161202    0.003344       NaN   
3     CONSENSUS          1  315.842665  407.238863    0.000433       NaN   
4     CONSENSUS          1  151.219209  328.139062    0.001450       NaN   
...         ...        ...         ...         ...         ...       ...   
6682  CONSENSUS          1  144.461281  450.255977    0.000009       NaN   
6683  CONSENSUS          1  364.670106  380.218136    0.000024       NaN   
6684  CONSENSUS          1  283.771130  482.272256    0.000082       NaN   
6685  CONSENSUS          1  343.010734  399.223823    0.000128       NaN   
6686  CONSENSUS          1  245.589066  361.207980    0.000035       NaN   

      intensity_0  intensity_1  intensity_2  intensity_3  ...  intensity_38  \
0       